# 3. Modelos

En este apartado se aplicarán diferentes modelos de ML vistos en clase. Para cada uno se estudiará la mejor combinación de parámetros, y a partir de ellos se construirá un nuevo modelo sobre el cuál se realizarán las predicciones.

Antes de comezar a crear y entrenar modelos, se ha de crear un pipeline que permita implementar de manera eficiente una consecución de transformaciones a los datos. Posteriormente, se ha de convertir el dataset ya transformado a un formato interpretable por la librería ml.

Finalmente se probarán técnicas de ensemble de modelos (modelos de segundo orden) y se compararán con los resultados con los modelos de primer orden.

## Inicialización de Spark y carga de librerías

In [20]:
import sys
import os

spark_path = "/Applications/spark-2.1.0-bin-hadoop2.7"
sys.path.append(spark_path + "/python")
sys.path.append(spark_path + "/python/lib/py4j-0.10.4-src.zip")

from pyspark.sql import SparkSession

spark = SparkSession.builder.master("local[*]").appName("Entrega") \
    .getOrCreate()

spark

In [21]:
from pyspark.ml.feature import StringIndexer, IndexToString, VectorAssembler, OneHotEncoder, StandardScaler
from pyspark.ml.regression import RandomForestRegressor, LinearRegression, GBTRegressor
from pyspark.sql.functions import col, count, mean, sum as agg_sum
from pyspark.ml.evaluation import Evaluator
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder
from pyspark.sql.functions import lit, log1p, expm1, udf 
from pyspark.sql.types import IntegerType, DoubleType 
from pyspark.mllib.stat import Statistics 
from pyspark.mllib.linalg import Vectors
from pyspark.sql.functions import avg
from pyspark.ml import Pipeline
from math import sqrt

import time
import pandas as pd
import numpy as np

seed = 2017

## Creación de la función de evaluación

In [42]:
class UserEval(Evaluator):
    '''
    When a userID is predicted when it is not already trained (all userID  data is used on validation 
    group and none of them to train), prediction is nan,  so RegressionEvaluator returns Nan.
    To solve this we must change RegressionEvaluator by MiValidacion
    '''
    def __init__(self,predictionCol='prediction', targetCol='label'):        
        super(UserEval, self).__init__()
        self.predictionCol=predictionCol
        self.targetCol=targetCol
        
    def _evaluate(self, dataset):       
        error=self.rmsle(dataset,self.predictionCol,self.targetCol)
        return error
    
    def isLargerBetter(self):
        return False
    
    @staticmethod
    def rmsle(dataset,predictionCol,targetCol):
        return sqrt(dataset.select(avg((log1p(dataset[targetCol]) - log1p(dataset[predictionCol])) ** 2)).first()[0])
 
                                   
evaluator = UserEval()

## Lectura de datos

Se importarán los datos preprocesados en el notebook anterior, y se aplicará un pipiline con las tranformaciones necesarias crear variables dummies, escalar los datos y pasarlos al formato adecuado para aplicar los modelos de ML.

In [101]:
path = "preprocessed"
datos = spark.read.csv(path, header=True, inferSchema=True, nullValue= 'NA')

## Creación del pipeline de transformaciones

Dado que los modelos no tabajan con variables categóricas, se hace necesaria su transformación a variables Dummies. Se trabajará con las librerías que proporciona el paquete ml de spark, implementándolas en un pipeline que permite concatenar operaciones sobre el conjunto de datos de forma más eficiente que realizando las transformaciones una a una

In [102]:
# División entre variables categóricas y numéricas
categ = [tupla[0] for tupla in datos.dtypes if tupla[1] == "string"]
no_categ = [tupla[0] for tupla in datos.dtypes if tupla[1] != "string"]

no_categ.remove('SalePrice')
no_categ.remove('Id')

# A cada columna categórica se le asocia un StringIndexer
StringIndex={c:StringIndexer(inputCol=c, outputCol=c+'_n') for c in categ}

# Codificación One Hot sobre  la salida StringIndex
encoders={c:OneHotEncoder(inputCol = c + '_n', outputCol = c + '_dummies') for c in categ}

# Union de las columnas en una de tipo vector (formato requerido por los modelos de ml).
assembler = VectorAssembler(inputCols=[c+'_dummies' for c in categ] + no_categ, outputCol="features")

# Escalado de los datos
standard_scaler = StandardScaler(withMean=True, withStd=True, inputCol='features', outputCol='scaledFeatures')


# Se crea un pipeline que permite concatenar los estimadores creados anteriormente
piplist=list(StringIndex.values())+list(encoders.values())+[assembler]+[standard_scaler]
pip=Pipeline(stages=piplist)

# Se entrena el transforamdor
pippre=pip.fit(datos)

In [103]:
# Separación del conjunto en train y test
test = datos.filter(datos.SalePrice.isNull())
train = datos.filter(datos.SalePrice.isNotNull())

print("El tamaño del conjunto de train es : ", train.select('Id').count())
print("El tamaño del conjunto de test es : ", test.select('Id').count())

El tamaño del conjunto de train es :  1460
El tamaño del conjunto de test es :  1459


In [104]:
# Aplicación del pipeline al conjunto de test
train_trans = pippre.transform(train)

#features contiene variables predictoras, label contiene variable objetivo.
train_trans = train_trans.select('features', train_trans.SalePrice.alias('label'),'Id')

train_trans.show(5)

+--------------------+------+---+
|            features| label| Id|
+--------------------+------+---+
|(259,[0,4,5,7,10,...|208500|  1|
|(259,[0,4,5,7,10,...|181500|  2|
|(259,[0,4,5,8,10,...|223500|  3|
|(259,[0,4,5,8,10,...|140000|  4|
|(259,[0,4,5,8,10,...|250000|  5|
+--------------------+------+---+
only showing top 5 rows



In [105]:
# Aplicación del pipeline al conjunto de test
test_trans = pippre.transform(test)

#features contiene variables predictoras, label contiene variable objetivo.
test_trans = test_trans.select('features',test_trans.SalePrice.alias('label'),'Id')

test_trans.show(5)

+--------------------+-----+----+
|            features|label|  Id|
+--------------------+-----+----+
|(259,[3,4,5,7,10,...| null|1461|
|(259,[0,4,5,8,10,...| null|1462|
|(259,[0,4,5,8,10,...| null|1463|
|(259,[0,4,5,8,10,...| null|1464|
|(259,[0,4,5,8,11,...| null|1465|
+--------------------+-----+----+
only showing top 5 rows



## Separación de los datos de train_trans en train y validation

Como no se tiene información acerca de SalePrice en los datos de test, hay que hacer una división del conjunto de entrenamiento train_trans en validación y train. Esto nos permitirá ajustar los modelos y obtener una estimación del error cometido por éstos.

In [59]:
train, validation = train_trans.randomSplit([0.75, 0.25], seed=seed)

## Modelos de primer orden

### Random Forest

In [45]:
# Instanciación del modelo
rf_1 = RandomForestRegressor(predictionCol='p_rf_1', seed=seed)

# Definición del grid
paramGrid = ParamGridBuilder() \
    .addGrid(rf_1.minInstancesPerNode, [5, 10, 15]) \
    .addGrid(rf_1.maxDepth, [9, 11, 15]) \
    .addGrid(rf_1.numTrees, [100, 125, 175]) \
    .build()
    
# Instanciación del método de crosvalidación
crossval = CrossValidator(estimator=rf_1,
                          estimatorParamMaps=paramGrid,
                          evaluator=UserEval(predictionCol='p_rf_1'),
                          numFolds=5, 
                          seed=seed) 


t0 = time.time()
rf_1_cv_mod = crossval.fit(train)
tt = time.time() - t0
print('Tiempo ajuste de parámetros {} min.'.format(round(tt/60,2)))

# Tabla results
par_map = rf_1_cv_mod.getEstimatorParamMaps()
lpars = [{par.name: value for par, value in par_comb.items()} for par_comb in par_map]

pars_df = pd.DataFrame(lpars)
pars_df['score'] = rf_1_cv_mod.avgMetrics
pars_df.sort_values(by='score', ascending=False)

Tiempo ajuste de parámetros 27.18 min.


,maxDepth,minInstancesPerNode,numTrees,score
18,9,15,100,0.159608
21,11,15,100,0.159396
24,15,15,100,0.159390
19,9,15,125,0.159207
22,11,15,125,0.158989
25,15,15,125,0.158969
20,9,15,175,0.158794
23,11,15,175,0.158609
26,15,15,175,0.158607
10,9,10,125,0.155544


In [71]:
# Se aplica el modelo con los mejores parámetros sobre el conjunto de validación
val = rf_1_cv_mod.bestModel.transform(validation)
val.show(5)
evaluator = UserEval(predictionCol='p_rf_1')
modelos_eval = pd.DataFrame([('rf1',evaluator.evaluate(val))],columns=['Modelo','Score'])
modelos_eval

+--------------------+------+----+------------------+
|            features| label|  Id|            p_rf_1|
+--------------------+------+----+------------------+
|(259,[0,4,5,7,10,...|142000| 420| 141066.5900335923|
|(259,[0,4,5,7,10,...|117500| 932|123437.54053191554|
|(259,[0,4,5,7,10,...|116050|1201|112303.12150070831|
|(259,[0,4,5,7,10,...|145000| 255|144611.78038321994|
|(259,[0,4,5,7,10,...|140000| 675|149498.42265306116|
+--------------------+------+----+------------------+
only showing top 5 rows



,Modelo,Score
0,rf1,0.147791


### Gradient Bosting

In [47]:
# Instanciación del modelo
gbt_1 = GBTRegressor(predictionCol='p_gbt_1',featuresCol='features', seed=seed)

# Definición del grid
paramGrid = ParamGridBuilder() \
    .addGrid(gbt_1.minInstancesPerNode , [20,25,30,40]) \
    .addGrid(gbt_1.maxDepth, [2,3,4]) \
    .addGrid(gbt_1.maxIter , [50,100]) \
    .build()
    
    
# Instanciación del método de crosvalidación
crossval = CrossValidator(estimator=gbt_1,
                          estimatorParamMaps=paramGrid,
                          evaluator=UserEval(predictionCol='p_gbt_1'),
                          numFolds=5, 
                          seed=seed) 

t0 = time.time()
gbt_cv_mod = crossval.fit(train)
tt = time.time() - t0
print('Tiempo ajuste de parámetros {} min.'.format(round(tt/60,2)))

# Tabla results
par_map = gbt_cv_mod.getEstimatorParamMaps()
lpars = [{par.name: value for par, value in par_comb.items()} for par_comb in par_map]
pars_df = pd.DataFrame(lpars)
pars_df['score'] = gbt_cv_mod.avgMetrics
pars_df.sort_values(by='score', ascending=False)

Tiempo ajuste de parámetros 50.55 min.


,maxDepth,maxIter,minInstancesPerNode,score
22,4,50,40,0.164788
0,2,50,20,0.163046
16,4,50,20,0.162905
20,4,50,30,0.162541
18,4,50,25,0.162345
8,3,50,20,0.161049
4,2,50,30,0.160535
23,4,100,40,0.160266
12,3,50,30,0.159542
2,2,50,25,0.159412


In [72]:
val=gbt_cv_mod.bestModel.transform(val)
val.show(5)
evaluator=UserEval(predictionCol='p_gbt_1')
modelos_eval=pd.concat([modelos_eval,pd.DataFrame([('gbt',evaluator.evaluate(val))],columns=['Modelo','Score'])])
modelos_eval

+--------------------+------+----+------------------+------------------+
|            features| label|  Id|            p_rf_1|           p_gbt_1|
+--------------------+------+----+------------------+------------------+
|(259,[0,4,5,7,10,...|142000| 420| 141066.5900335923|138293.20013792816|
|(259,[0,4,5,7,10,...|117500| 932|123437.54053191554| 119590.9042959555|
|(259,[0,4,5,7,10,...|116050|1201|112303.12150070831|107560.65555723177|
|(259,[0,4,5,7,10,...|145000| 255|144611.78038321994|138073.96851261955|
|(259,[0,4,5,7,10,...|140000| 675|149498.42265306116| 143671.2023690182|
+--------------------+------+----+------------------+------------------+
only showing top 5 rows



,Modelo,Score
0,rf1,0.147791
0,gbt,0.140176


### Logistic Regression

In [50]:
# Instanciación del modelo
lr_1 = lr = LinearRegression(predictionCol='p_lr_1',featuresCol='features')

# Definición del grid
paramGrid = ParamGridBuilder() \
    .addGrid(lr.regParam , [0, 0.001, 0.01, 0.05, 0.1, 0.15, 0.2]) \
    .addGrid(lr.elasticNetParam, [0, 0.05, 0.1, 0.15, 0.2, 0.25, 0.3, 0.8, 0.9]) \
    .build()
    
    
# Instanciación del método de crosvalidación
crossval = CrossValidator(estimator=lr_1,
                          estimatorParamMaps=paramGrid,
                          evaluator=UserEval(predictionCol='p_lr_1'),
                          numFolds=5, 
                          seed=seed) 

t0 = time.time()
lr_cv_mod = crossval.fit(train)
tt = time.time() - t0
print('Tiempo ajuste de parámetros {} min.'.format(round(tt/60,2)))

# Tabla results
par_map = lr_cv_mod.getEstimatorParamMaps()
lpars = [{par.name: value for par, value in par_comb.items()} for par_comb in par_map]
pars_df = pd.DataFrame(lpars)
pars_df['score'] = lr_cv_mod.avgMetrics
pars_df.sort_values(by='score', ascending=False)

Tiempo ajuste de parámetros 15.14 min.


,elasticNetParam,regParam,score
42,0.30,0.100,0.206933
56,0.10,0.200,0.206919
49,0.20,0.150,0.206670
48,0.15,0.150,0.206593
41,0.25,0.100,0.206590
40,0.20,0.100,0.206394
50,0.25,0.150,0.204048
35,0.90,0.050,0.202457
51,0.30,0.150,0.202263
34,0.80,0.050,0.202194


In [73]:
val=lr_cv_mod.bestModel.transform(val)
val.show(5)
evaluator=UserEval(predictionCol='p_lr_1')
modelos_eval=pd.concat([modelos_eval,pd.DataFrame([('lr',evaluator.evaluate(val))],columns=['Modelo','Score'])])
modelos_eval

+--------------------+------+----+------------------+------------------+------------------+
|            features| label|  Id|            p_rf_1|           p_gbt_1|            p_lr_1|
+--------------------+------+----+------------------+------------------+------------------+
|(259,[0,4,5,7,10,...|142000| 420| 141066.5900335923|138293.20013792816|130438.03970963112|
|(259,[0,4,5,7,10,...|117500| 932|123437.54053191554| 119590.9042959555|125003.23966162186|
|(259,[0,4,5,7,10,...|116050|1201|112303.12150070831|107560.65555723177| 89372.92642578483|
|(259,[0,4,5,7,10,...|145000| 255|144611.78038321994|138073.96851261955|138350.07047059387|
|(259,[0,4,5,7,10,...|140000| 675|149498.42265306116| 143671.2023690182| 135955.3524235445|
+--------------------+------+----+------------------+------------------+------------------+
only showing top 5 rows



,Modelo,Score
0,rf1,0.147791
0,gbt,0.140176
0,lr,0.190052


## Modelos de segundo orden

## Ensemble de los modelos anteriores

In [75]:
# Se entrena el modelo superior sobre el conjunto de entrenamiento concatenando las predicciones de cada modelo
tmp = lr_cv_mod.bestModel.transform(train)
tmp = rf_1_cv_mod.bestModel.transform(tmp)
tmp = gbt_cv_mod.bestModel.transform(tmp)
conjunto = VectorAssembler(inputCols=["p_lr_1", "p_rf_1", "p_gbt_1"], outputCol="features_ensemble")

# Se pasan a formato vecto
train_ensemble=conjunto.transform(tmp)
validation_ensemble=conjunto.transform(val)


# Regresión lineal de los modelos
lr_ensemble = LinearRegression(predictionCol='p_ensemble',featuresCol='features_ensemble')
lr_ensemble=lr_ensemble.fit(train_ensemble)

# Evaluación sobre el conjunto de test
validation_ensemble=lr_ensemble.transform(validation_ensemble)
evaluator=UserEval(predictionCol='p_ensemble')
modelos_eval=pd.concat([modelos_eval,pd.DataFrame([('ensemble',evaluator.evaluate(validation_ensemble))]\
                                                  ,columns=['Modelo','Score'])])

modelos_eval

,Modelo,Score
0,rf1,0.147791
0,gbt,0.140176
0,lr,0.190052
0,ensemble,0.178200


## Subida a Kaggle

Por último se realizarán las predicciones sobre el conjunto de datos de test que ha sido transformado al inicio del ejercicio. Para ello se usará el modelo con mejor score.

In [106]:
test_kaggle=gbt_cv_mod.bestModel.transform(test_trans)
test_kaggle.show(10)

+--------------------+-----+----+------------------+
|            features|label|  Id|           p_gbt_1|
+--------------------+-----+----+------------------+
|(259,[3,4,5,7,10,...| null|1461|125719.39819980791|
|(259,[0,4,5,8,10,...| null|1462|158099.75462175196|
|(259,[0,4,5,8,10,...| null|1463| 190505.7716740466|
|(259,[0,4,5,8,10,...| null|1464| 188885.9425430358|
|(259,[0,4,5,8,11,...| null|1465|202197.43907302304|
|(259,[0,4,5,8,10,...| null|1466| 176094.7158149631|
|(259,[0,4,5,8,10,...| null|1467|162724.53323682692|
|(259,[0,4,5,8,10,...| null|1468| 165423.4991657976|
|(259,[0,4,5,7,10,...| null|1469|195813.52680196438|
|(259,[0,4,5,7,10,...| null|1470|125383.46484855308|
+--------------------+-----+----+------------------+
only showing top 10 rows



In [111]:
submission=test_kaggle.select(['Id','p_gbt_1'])
submission=submission.toPandas()
submission.columns=['Id','SalePrice']
submission.to_csv('submission/submission.csv', sep=",",header=True,index=False)